In [151]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import seaborn as sns
from collections import defaultdict
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn import datasets, linear_model
from sklearn import neighbors
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [2]:
## Read the Key, logerror, transaction date set
df_parcel = pd.read_csv("c:/Users/ibipul/Documents/Python Scripts/datasets/zillow_data/train_2016_v2.csv")

In [3]:
## Read the attribute set
df_properties = pd.read_csv("c:/Users/ibipul/Documents/Python Scripts/datasets/zillow_data/properties_2016.csv",low_memory=False)

In [8]:
#Checking for missing data
for i in df_properties.columns:
    print(i, sum(df_properties[i].notnull()*1))

parcelid 2985217
airconditioningtypeid 811519
architecturalstyletypeid 6061
basementsqft 1628
bathroomcnt 2973755
bedroomcnt 2973767
buildingclasstypeid 12629
buildingqualitytypeid 1938488
calculatedbathnbr 2856305
decktypeid 17096
finishedfloor1squarefeet 202717
calculatedfinishedsquarefeet 2929652
finishedsquarefeet12 2709184
finishedsquarefeet13 7672
finishedsquarefeet15 190798
finishedsquarefeet50 202717
finishedsquarefeet6 22001
fips 2973780
fireplacecnt 312637
fullbathcnt 2856305
garagecarcnt 883267
garagetotalsqft 883267
hashottuborspa 69014
heatingorsystemtypeid 1806401
latitude 2973780
longitude 2973780
lotsizesquarefeet 2709118
poolcnt 517534
poolsizesum 27960
pooltypeid10 36939
pooltypeid2 32075
pooltypeid7 485459
propertycountylandusecode 2972940
propertylandusetypeid 2973780
propertyzoningdesc 1978629
rawcensustractandblock 2973780
regionidcity 2922372
regionidcounty 2973780
regionidneighborhood 1156402
regionidzip 2971237
roomcnt 2973742
storytypeid 1624
threequarterbathn

In [30]:
# #Replace Nans in these variable values with Zeroes and hot encode the numeric values
categorical_variables=['airconditioningtypeid','buildingqualitytypeid','heatingorsystemtypeid']
## Impute the following with median values
numeric_variables = ['bathroomcnt','bedroomcnt','roomcnt','fullbathcnt', 'calculatedbathnbr',
          'calculatedfinishedsquarefeet', 'finishedsquarefeet12','garagetotalsqft','yearbuilt',
          'structuretaxvaluedollarcnt','taxvaluedollarcnt', 'landtaxvaluedollarcnt']
# This is the key variable that will help with the join
key_variable = ['parcelid']
# Full variable list:
properties_variables = key_variable + numeric_variables + categorical_variables

In [32]:
## Sub setting the properties variables
df = df_properties[properties_variables]

In [104]:
## Merging the id,logerr,transaction data set with the above
dfmain = pd.merge(df_parcel,df,on='parcelid')

In [105]:
# date parser
def get_month(transaction_date):
    s=transaction_date.split('-')
    month=int(s[1])
    return month

# Adding a new column as month of transaction
dfmain['month'] = dfmain['transactiondate'].apply(lambda x: get_month(x))

In [106]:
dfmain.shape

(90275, 19)

In [107]:
## Impute numeric variables
#Checking for missing data
for i in numeric_variables:
    print(i, sum(dfmain[i].notnull()*1))

bathroomcnt 90275
bedroomcnt 90275
roomcnt 90275
fullbathcnt 89093
calculatedbathnbr 89093
calculatedfinishedsquarefeet 89614
finishedsquarefeet12 85596
garagetotalsqft 29937
yearbuilt 89519
structuretaxvaluedollarcnt 89895
taxvaluedollarcnt 90274
landtaxvaluedollarcnt 90274


In [108]:
#Creating an imputation dictionary
imputation_dict = defaultdict()
for i in numeric_variables:
    x = sum(dfmain[i].notnull()*1)
    #print(i, x)
    if (90275 -x) !=0:
        imputation_dict[i]=dfmain[i].median()

In [109]:
imputation_dict

defaultdict(None,
            {'calculatedbathnbr': 2.0,
             'calculatedfinishedsquarefeet': 1540.0,
             'finishedsquarefeet12': 1518.0,
             'fullbathcnt': 2.0,
             'garagetotalsqft': 433.0,
             'landtaxvaluedollarcnt': 192970.0,
             'structuretaxvaluedollarcnt': 132000.0,
             'taxvaluedollarcnt': 342872.0,
             'yearbuilt': 1970.0})

In [110]:
# Imputing missing data with median values of columns
for i in numeric_variables:
    x = sum(dfmain[i].isnull()*1)
    if x>0:
        dfmain[i].replace(np.nan,imputation_dict[i],inplace=True)
        print("imputation happened for: ", i," with ",imputation_dict[i])

imputation happened for:  fullbathcnt  with  2.0
imputation happened for:  calculatedbathnbr  with  2.0
imputation happened for:  calculatedfinishedsquarefeet  with  1540.0
imputation happened for:  finishedsquarefeet12  with  1518.0
imputation happened for:  garagetotalsqft  with  433.0
imputation happened for:  yearbuilt  with  1970.0
imputation happened for:  structuretaxvaluedollarcnt  with  132000.0
imputation happened for:  taxvaluedollarcnt  with  342872.0
imputation happened for:  landtaxvaluedollarcnt  with  192970.0


In [111]:
dfmain.head()

,parcelid,logerror,transactiondate,bathroomcnt,bedroomcnt,roomcnt,fullbathcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,garagetotalsqft,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,airconditioningtypeid,buildingqualitytypeid,heatingorsystemtypeid,month
0,11016594,0.0276,2016-01-01,2.0,3.0,0.0,2.0,2.0,1684.0,1684.0,433.0,1959.0,122754.0,360170.0,237416.0,1.0,4.0,2.0,1
1,14366692,-0.1684,2016-01-01,3.5,4.0,0.0,3.0,3.5,2263.0,2263.0,468.0,2014.0,346458.0,585529.0,239071.0,NaN,NaN,NaN,1
2,12098116,-0.0040,2016-01-01,3.0,2.0,0.0,3.0,3.0,2217.0,2217.0,433.0,1940.0,61994.0,119906.0,57912.0,1.0,4.0,2.0,1
3,12643413,0.0218,2016-01-02,2.0,2.0,0.0,2.0,2.0,839.0,839.0,433.0,1987.0,171518.0,244880.0,73362.0,1.0,4.0,2.0,1
4,14432541,-0.0050,2016-01-02,2.5,4.0,8.0,2.0,2.5,2283.0,2283.0,598.0,1981.0,169574.0,434551.0,264977.0,NaN,NaN,NaN,1


In [112]:
# Checking for NaNs in categorical numeric variables
for i in categorical_variables:
    print(i, sum(dfmain[i].notnull()*1))


airconditioningtypeid 28781
buildingqualitytypeid 57364
heatingorsystemtypeid 56080


In [113]:
## Replacing NaNs in Categorical variables with 0's
for i in categorical_variables:
    dfmain[i].replace(np.nan,0.0,inplace=True)

In [114]:
# Checking for NaNs in categorical numeric variables
for i in categorical_variables:
    print(i, sum(dfmain[i].notnull()*1))

airconditioningtypeid 90275
buildingqualitytypeid 90275
heatingorsystemtypeid 90275


In [115]:
## One hot encoding the categorical variables
columnsToEncode = categorical_variables
# Adding dummies
for feature in columnsToEncode:
        print('One Hot Encoding: ', feature)
        dfmain = pd.concat([dfmain, pd.get_dummies(dfmain[feature],prefix=feature)], axis=1)
        del dfmain[feature]


One Hot Encoding:  airconditioningtypeid
One Hot Encoding:  buildingqualitytypeid
One Hot Encoding:  heatingorsystemtypeid


In [116]:
dfmain.head()

,parcelid,logerror,transactiondate,bathroomcnt,bedroomcnt,roomcnt,fullbathcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,heatingorsystemtypeid_6.0,heatingorsystemtypeid_7.0,heatingorsystemtypeid_10.0,heatingorsystemtypeid_11.0,heatingorsystemtypeid_12.0,heatingorsystemtypeid_13.0,heatingorsystemtypeid_14.0,heatingorsystemtypeid_18.0,heatingorsystemtypeid_20.0,heatingorsystemtypeid_24.0
0,11016594,0.0276,2016-01-01,2.0,3.0,0.0,2.0,2.0,1684.0,1684.0,...,0,0,0,0,0,0,0,0,0,0
1,14366692,-0.1684,2016-01-01,3.5,4.0,0.0,3.0,3.5,2263.0,2263.0,...,0,0,0,0,0,0,0,0,0,0
2,12098116,-0.0040,2016-01-01,3.0,2.0,0.0,3.0,3.0,2217.0,2217.0,...,0,0,0,0,0,0,0,0,0,0
3,12643413,0.0218,2016-01-02,2.0,2.0,0.0,2.0,2.0,839.0,839.0,...,0,0,0,0,0,0,0,0,0,0
4,14432541,-0.0050,2016-01-02,2.5,4.0,8.0,2.0,2.5,2283.0,2283.0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
dfmain.columns

Index(['parcelid', 'logerror', 'transactiondate', 'bathroomcnt', 'bedroomcnt',
       'roomcnt', 'fullbathcnt', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'garagetotalsqft', 'yearbuilt', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'month',
       'airconditioningtypeid_0.0', 'airconditioningtypeid_1.0',
       'airconditioningtypeid_3.0', 'airconditioningtypeid_5.0',
       'airconditioningtypeid_9.0', 'airconditioningtypeid_11.0',
       'airconditioningtypeid_13.0', 'buildingqualitytypeid_0.0',
       'buildingqualitytypeid_1.0', 'buildingqualitytypeid_4.0',
       'buildingqualitytypeid_6.0', 'buildingqualitytypeid_7.0',
       'buildingqualitytypeid_8.0', 'buildingqualitytypeid_10.0',
       'buildingqualitytypeid_11.0', 'buildingqualitytypeid_12.0',
       'heatingorsystemtypeid_0.0', 'heatingorsystemtypeid_1.0',
       'heatingorsystemtypeid_2.0', 'heatingorsystemtypeid_6.0',
       'heatingors

In [121]:
## Creating a test-train split
train=dfmain.sample(frac=0.75,random_state=200)
test=dfmain.drop(train.index)

In [122]:
train.shape

(67706, 45)

In [123]:
test.shape

(22569, 45)

In [148]:
# Separating predictors and response variables
all_column_names = list(train.columns)
for i in ['parcelid','logerror','transactiondate']:
    all_column_names.remove(i)
# X variables
x_vars = all_column_names
y_var = 'logerror'
# Train set
train_Y = train[y_var]
train_X = train[x_vars]
#Test set
test_Y = train[y_var]
test_X = train[x_vars]

In [153]:
#Create a K-Nearest Neighbour model
knn = neighbors.KNeighborsRegressor(n_neighbors=5)
# Train the model using the training sets & Make predictions using the testing set
logerror_prediction_y = knn.fit(train_X, train_Y).predict(test_X)

In [154]:
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(test_Y, logerror_prediction_y))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(test_Y, logerror_prediction_y))

Mean squared error: 0.02
Variance score: 0.21
